In [1]:
import pandas as pd
import numpy as np

In [2]:
sales = pd.read_csv("kc_house_data.csv")
testing = pd.read_csv('wk3_kc_house_test_data.csv')
training = pd.read_csv('wk3_kc_house_train_data.csv')

In [3]:
def get_numpy_data(data,features,output):
    constant_col = np.ones((len(data),1))
    features_matrix = np.hstack((constant_col, data.as_matrix(features)))
    output_array = data.as_matrix([output])
    return (features_matrix, output_array)

In [4]:
def predict_outcome(features_matrix,weight):
    return np.dot(features_matrix,weight).reshape((-1,1))

In [5]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    return (normalized_features,norms)

In [6]:
(features_matrix,output) = get_numpy_data(sales,['sqft_living','bedrooms'],'price')
(normalized_features,norms) = normalize_features(features_matrix)
initial_weights = [1,4,1]
initial_predictions = predict_outcome(normalized_features,initial_weights)
# print normalized_features
ro = [1,4,1]
feature1 = np.reshape(normalized_features[:,1],(-1,1))
feature2 = np.reshape(normalized_features[:,2],(-1,1))
ro[1] = (feature1*(output - initial_predictions + initial_weights[1]*feature1)).sum()
ro[2] = (feature2*(output - initial_predictions + initial_weights[2]*feature2)).sum()
print ro


[1, 87939470.823251754, 80966698.66623947]


In [10]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_outcome(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    feature = np.reshape(feature_matrix[:,i],(-1,1))
    ro_i = (feature*(output - prediction + weights[i]*feature)).sum()
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [11]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([[1.], [1.]]), np.array([1., 4.]), 0.1)

0.425558846691


In [12]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    diff_i = []
    n_of_features = feature_matrix.shape[1]
    while diff_i == [] or sum(diff_i) >= tolerance:
        diff_i = []
        for i in range(0,n_of_features):
            new_weight_i = lasso_coordinate_descent_step(i,feature_matrix,output,weights,l1_penalty)
            diff_i.append(abs(new_weight_i - weights[i]))
            weights[i] = new_weight_i
    return weights

In [13]:
lasso_weights = lasso_cyclical_coordinate_descent(normalized_features,output,[0,0,0],1e7,1.0)
print lasso_weights

[21624996.134590954, 63157248.87730661, 0.0]


In [14]:
((output - predict_outcome(normalized_features, lasso_weights))**2).sum()

1630492460021214.0

# Evaluate Lasso with more features

In [15]:
(advanced_features_matrix,training_output) = get_numpy_data(training,['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated'],'price')

In [16]:
(advanced_normalized_features,advanced_norms) = normalize_features(advanced_features_matrix)

In [17]:
weights1e7 = lasso_cyclical_coordinate_descent(advanced_normalized_features,training_output,np.zeros(14),1e7,1)
print weights1e7

[ 23864690.05262006         0.                 0.          30495550.44912805
         0.                 0.           1901633.6337427    5705764.85465323
         0.                 0.                 0.                 0.
         0.                 0.        ]


In [18]:
weights1e8 = lasso_cyclical_coordinate_descent(advanced_normalized_features,training_output,np.zeros(14),1e8,1.0)
print weights1e8

[ 53621004.68971471         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]


In [19]:
weights1e4 = lasso_cyclical_coordinate_descent(advanced_normalized_features,training_output,np.zeros(14),1e4,5e5)
print weights1e4

[  7.82190763e+07  -1.20984097e+07   1.80739372e+06   1.23989425e+08
  -1.26486734e+06  -4.01783338e+05   5.40258988e+06   4.74063237e+06
   1.35859466e+07   6.12220003e+07  -6.57557829e+07  -1.74916212e+07
  -1.34202040e+08   2.27238892e+06]


## Normalize weights

In [21]:
normalized_weights1e4 = weights1e4 / advanced_norms
normalized_weights1e7 = weights1e7 / advanced_norms
normalized_weights1e8 = weights1e8 / advanced_norms
# should print 161.31745624837794
print normalized_weights1e7[3]

135.17652673


# compare RSS on different models

In [22]:
def calculate_RSS(output, features,normalized_weights):
    return ((output - predict_outcome(features, normalized_weights))**2).sum()

(test_features_matrix,testing_output) = get_numpy_data(testing,['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated'],'price')
print calculate_RSS(testing_output,test_features_matrix,normalized_weights1e4)
print calculate_RSS(testing_output,test_features_matrix,normalized_weights1e7)
print calculate_RSS(testing_output,test_features_matrix,normalized_weights1e8)

1.13610819768e+14
1.63103559598e+14
2.8471892521e+14
